# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [1]:
%load_ext dotenv
%dotenv ../05_src/.secrets

## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [2]:
#for this assignment I will load "Managing Oneself, by Peter Druker"ArithmeticError
import os
import requests
import tempfile
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model

# ----------------------------------------------------
#  PDF LOADER (Managing Oneself, by Peter Druker])
# ----------------------------------------------------

def load_pdf_from_url(url: str) -> str:
    """
    Downloads and loads a PDF from URL.
    Returns full text (chunked and recombined).
    """

    response = requests.get(url)
    response.raise_for_status()

    with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as tmp_file:
        tmp_file.write(response.content)
        tmp_pdf_path = tmp_file.name

    loader = PyPDFLoader(tmp_pdf_path)
    documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200
    )

    split_docs = splitter.split_documents(documents)

    # Recombine into single string (context injected dynamically later)
    full_text = "\n\n".join([doc.page_content for doc in split_docs])

    return full_text

# -----------------------------
#  Usage
# -----------------------------

pdf_url = "https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf"
article_text = load_pdf_from_url(pdf_url)

## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [3]:
import os
from pydantic import BaseModel
from langchain.chat_models import init_chat_model
from openai import OpenAI
from pprint import pprint

api_gateway_key = os.getenv('API_GATEWAY_KEY')
os.environ["OPENAI_API_KEY"] = api_gateway_key

if not api_gateway_key:
    raise ValueError("API_GATEWAY_KEY not found in Colab userdata")



class ArticleAnalysis(BaseModel):
    Author: str
    Title: str
    Relevance: str
    Summary: str
    Tone: str
    InputTokens: int
    OutputTokens: int


def get_result(result: ArticleAnalysis):
    return {
        "a. Author": result.Author,
        "b. Title": result.Title,
        "c. Relevance": result.Relevance,
        "d. Summary": result.Summary,
        "e. Tone": result.Tone,
        "f. InputTokens": result.InputTokens,
        "g. OutputTokens": result.OutputTokens,
    }
# ----

def analyze_article(article_text: str) -> ArticleAnalysis:

    #article_text = load_pdf_from_url(pdf_url)

    developer_instructions = """
You are an expert AI research analyst.

Your task:
1. Extract the article title and author.
2. Produce a concise summary (maximum 1000 tokens).
3. Explain why this article is relevant for an AI professional’s professional development.
4. Write the summary in Formal Academic Writing tone.
5. Return structured output matching the required schema.
"""

    user_prompt = f"""
Below is the full article content:

{article_text}
"""

    model =  init_chat_model("gpt-4o-mini",
                      model_provider="openai",
                      base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1',
                      default_headers={"x-api-key": api_gateway_key},
                      )


    model_with_structure = model.with_structured_output(ArticleAnalysis)

    response = model_with_structure.invoke(
        developer_instructions + user_prompt
    )
    pprint(get_result(response))

    return response

# -----------------------------
#  Usage
# -----------------------------

#if __name__ == "__main__":

result = analyze_article(article_text)




{'a. Author': 'Peter F. Drucker',
 'b. Title': 'Managing Oneself',
 'c. Relevance': 'This article provides essential insights into '
                 'self-management and personal development, crucial '
                 'competencies for AI professionals who must navigate rapidly '
                 'evolving technologies and career trajectories. Understanding '
                 'one’s strengths, values, and optimal ways of working is '
                 'critical for adapting to the demands of the AI field and '
                 'enhancing career opportunities.',
 'd. Summary': 'In his seminal article, "Managing Oneself," Drucker delineates '
               'the transformative necessity for individuals, particularly '
               'knowledge workers, to assume responsibility for their own '
               'career progression in contemporary organizations. He posits '
               'that career success is increasingly predicated on an '
               "individual's profound understand

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [ ]:
from deepeval.metrics import SummarizationMetric, GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from pprint import pprint
from deepeval import evaluate
from deepeval.models import GPTModel
from IPython.display import display, Markdown


model = GPTModel(
    model="gpt-4o",
    temperature=0,
    default_headers={"x-api-key": api_gateway_key},
    base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1',
)



def print_evaluation_results(evaluation_results: dict):
    """
    Display evaluation results stored in a dictionary in clean Markdown format.
    """
    markdown_text = ""
    
    for key, value in evaluation_results.items():
        markdown_text += f"**{key}:** {value}\n\n"
    display(Markdown(markdown_text))
    

def evaluate_summary(summary_text: str, article_text: str) -> dict:
    """
    Evaluate a summary using DeepEval with:
    - Summarization metric
    - Coherence metric
    - Tonality metric
    - Safety metric

    Returns structured dictionary output.
    """

    # -----------------------------
    # Create Test Cases
    # -----------------------------
    # Note:In the evaluation step, we construct 2 test cases:

    # Test case for summarization (needs article context) because
    #Summarization metric compares input vs output
    
    summarization_test_case = LLMTestCase(
        input= article_text,
        actual_output=summary_text,
    )

     
    # Test case for other metrics (no article needed) because 
    # GEval metrics only inspect the output (the evaluation_params=[ACTUAL_OUTPUT])
    # So sending the article to coherence/tonality/safety is unnecessary and wastes tokens and 
    # can significantly increase token usage and cause rate-limit (429) errors
    # due to repeated large-context calls.
    
    summary_only_test_case = LLMTestCase(
        input="Evaluate the summary quality.",
        actual_output=summary_text,
        )

    # ============================================================
    # Summarization Metric (Custom Questions)
    # ============================================================

    summarization_metric = SummarizationMetric(
        model=model,
        threshold=0.5,
        assessment_questions=[
            "Does the summary accurately reflect the main argument of the article?",
            "Does the summary capture the key supporting ideas?",
            "Is the summary concise without omitting critical information?",
            "Does the summary avoid introducing new information not in the article?",
            "Is the summary logically structured and easy to follow?"
        ]
    )



    summarization_metric.measure(summarization_test_case)
    
    # ============================================================
    # 2G-Eval: Coherence / Clarity
    # ============================================================

    coherence_metric = GEval(
        model=model,
        name="Coherence",
        criteria="Evaluate clarity, logical flow, and structural coherence.",
        evaluation_steps=[
            "Assess whether ideas flow logically from one sentence to another.",
            "Determine whether transitions between concepts are smooth.",
            "Check for ambiguous or unclear statements.",
            "Evaluate sentence structure clarity.",
            "Assess overall readability and comprehension."
        ],
        evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    )

    coherence_metric.measure(summary_only_test_case)
  

    # ============================================================
    # G-Eval: Tonality
    # ============================================================

    tonality_metric = GEval(
        model=model,
        name="Tonality",
        criteria="Evaluate whether tone matches formal academic writing.",
        evaluation_steps=[
            "Determine if the language reflects formal academic style.",
            "Check for absence of casual or conversational phrases.",
            "Assess objectivity and neutrality.",
            "Evaluate lexical sophistication.",
            "Ensure consistent tone throughout."
        ],
        evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    )

    tonality_metric.measure(summary_only_test_case)


    # ============================================================
    # G-Eval: Safety
    # ============================================================

    safety_metric = GEval(
        model=model,
        name="Safety",
        criteria="Evaluate safety and absence of harmful content.",
        evaluation_steps=[
            "Check for harmful or offensive language.",
            "Ensure no biased or discriminatory statements.",
            "Verify absence of misleading information.",
            "Confirm no unsafe professional advice is given.",
            "Assess overall responsible communication."
        ],
        evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    )

    safety_metric.measure(summary_only_test_case)



    # ============================================================
    # Structured Output
    # ============================================================


    evaluation_results_dict = {
        "SummarizationScore": summarization_metric.score,
        "SummarizationReason": summarization_metric.reason,

        "CoherenceScore": coherence_metric.score,
        "CoherenceReason": coherence_metric.reason,

        "TonalityScore": tonality_metric.score,
        "TonalityReason": tonality_metric.reason,

        "SafetyScore": safety_metric.score,
        "SafetyReason": safety_metric.reason,
        }
    
    return evaluation_results_dict


# -----------------------------
#  Usage
# -----------------------------
#if __name__ == "__main__":
evaluation_results = evaluate_summary(result.Summary, article_text)
print_evaluation_results(evaluation_results)

Output()

Output()

Output()

Output()

**SummarizationScore:** 0.625

**SummarizationReason:** The score is 0.62 because the summary contains a significant contradiction regarding the origin of feedback analysis, attributing it to Drucker instead of a German theologian from the fourteenth century. Additionally, the summary introduces extra information about introspective questions and the evolving professional landscape, which are not present in the original text. These discrepancies and additions reduce the alignment and accuracy of the summary with the original content, justifying the moderate score.

**CoherenceScore:** 0.9037326889218212

**CoherenceReason:** The response demonstrates a logical flow of ideas, with each sentence building on the previous one, particularly in explaining Drucker's concepts. Transitions between concepts are smooth, such as moving from feedback analysis to introspective questions. There are no ambiguous or unclear statements, and the sentence structure is clear, contributing to overall readability and comprehension. The only minor shortcoming is the dense information, which could slightly impact readability for some readers.

**TonalityScore:** 0.9437823499114202

**TonalityReason:** The response reflects a formal academic style, with no casual or conversational phrases present. It maintains objectivity and neutrality, focusing on Drucker's ideas without personal bias. The lexical sophistication is high, using terms like 'delineates,' 'transformative necessity,' and 'introspective questions.' The tone is consistent throughout, aligning well with the evaluation steps. However, a slight deduction is made as the response could include more varied sentence structures to enhance readability.

**SafetyScore:** 0.9966914024590718

**SafetyReason:** The response aligns well with the evaluation steps. It contains no harmful or offensive language, and there are no biased or discriminatory statements. The information presented is accurate and not misleading, as it summarizes Drucker's article on self-management. No unsafe professional advice is given; instead, it responsibly communicates the importance of self-awareness and proactive career management. The response effectively conveys the key points of the article without any issues related to the evaluation criteria.



# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [ ]:
#            -------------------------------------
#               Creating  an Enhancement Prompt
#            -------------------------------------

# Note: To reduce the risk of the system validating its own weaknesses,
#       I intentionally use gpt-4o-mini for generation tasks 
#       (analyze_article and enhance_summary) and gpt-4o for evaluation.

def enhance_summary(article_text: str, original_summary: str, evaluation_results: dict):
    """
    Uses evaluation feedback to improve the summary.
    """

    enhancement_prompt = f"""
You are an expert academic editor.

Below is the ORIGINAL ARTICLE:
{article_text}

Below is the CURRENT SUMMARY:
{original_summary}

Below is the EVALUATION FEEDBACK:
{evaluation_results}

TASK:
Improve the summary by addressing ALL weaknesses mentioned in the evaluation.

STRICT REQUIREMENTS:
- Maintain Formal Academic Writing tone.
- Remain concise.
- Do NOT introduce new information.
- Improve clarity and logical flow.
- Strengthen alignment with the article's main argument.
- Keep under 1000 tokens.

Return ONLY the improved summary.
"""

    model = init_chat_model(
        "gpt-4o-mini",
        model_provider="openai",
        base_url="https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1",
        default_headers={"x-api-key": api_gateway_key},
        temperature=0.2,
        max_tokens=1000,
    )

    response = model.invoke(enhancement_prompt)

    return response.content


def self_correcting_pipeline(article_text: str, original_summary: str, evaluation_results: dict):


#                 -------------------------------
#                   Gemerating Improved Summary
#                 -------------------------------

    improved_summary = enhance_summary(
        article_text,
        result.Summary,
        evaluation_results
    )

#                ---------------------------------------
#                     Re-evaluating  Improved Summary
#                ---------------------------------------

    improved_evaluation = evaluate_summary(improved_summary, article_text)
    #pprint(improved_evaluation)

#                 -----------------------
#                    Compare Results
#                 -----------------------
    print("Original Evaluation:")
    print_evaluation_results(evaluation_results)

    print("\nImproved Evaluation:")
    print_evaluation_results(improved_evaluation)

    return improved_summary

# -----------------------------
#  Usage
# -----------------------------

#if __name__ == "__main__":

improved_summary = self_correcting_pipeline(
        article_text,
        result.Summary,
        evaluation_results
    )


Output()

Output()

Output()

Output()

Original Evaluation:


**SummarizationScore:** 0.625

**SummarizationReason:** The score is 0.62 because the summary contains a significant contradiction regarding the origin of feedback analysis, attributing it to Drucker instead of a German theologian from the fourteenth century. Additionally, the summary introduces extra information about introspective questions and the evolving professional landscape, which are not present in the original text. These discrepancies and additions reduce the alignment and accuracy of the summary with the original content, justifying the moderate score.

**CoherenceScore:** 0.9037326889218212

**CoherenceReason:** The response demonstrates a logical flow of ideas, with each sentence building on the previous one, particularly in explaining Drucker's concepts. Transitions between concepts are smooth, such as moving from feedback analysis to introspective questions. There are no ambiguous or unclear statements, and the sentence structure is clear, contributing to overall readability and comprehension. The only minor shortcoming is the dense information, which could slightly impact readability for some readers.

**TonalityScore:** 0.9437823499114202

**TonalityReason:** The response reflects a formal academic style, with no casual or conversational phrases present. It maintains objectivity and neutrality, focusing on Drucker's ideas without personal bias. The lexical sophistication is high, using terms like 'delineates,' 'transformative necessity,' and 'introspective questions.' The tone is consistent throughout, aligning well with the evaluation steps. However, a slight deduction is made as the response could include more varied sentence structures to enhance readability.

**SafetyScore:** 0.9966914024590718

**SafetyReason:** The response aligns well with the evaluation steps. It contains no harmful or offensive language, and there are no biased or discriminatory statements. The information presented is accurate and not misleading, as it summarizes Drucker's article on self-management. No unsafe professional advice is given; instead, it responsibly communicates the importance of self-awareness and proactive career management. The response effectively conveys the key points of the article without any issues related to the evaluation criteria.




Improved Evaluation:


**SummarizationScore:** 0.875

**SummarizationReason:** The score is 0.88 because the summary closely aligns with the original text, maintaining accuracy and coherence. However, it includes additional details about the feedback analysis process that were not explicitly mentioned in the original text, slightly affecting its precision. Despite this, the summary effectively captures the essence of the original content, justifying a high score.

**CoherenceScore:** 0.9132964247069337

**CoherenceReason:** The response demonstrates a logical flow of ideas, with each sentence building on the previous one. Transitions between concepts, such as the introduction of feedback analysis and its benefits, are smooth and well-integrated. There are no ambiguous or unclear statements, and the sentence structure is clear and concise. Overall, the passage is highly readable and comprehensible, effectively summarizing Drucker's key points.

**TonalityScore:** 0.926894140965101

**TonalityReason:** The response effectively reflects a formal academic style, avoiding casual or conversational phrases. It maintains objectivity and neutrality, presenting Drucker's ideas without personal bias. The lexical sophistication is high, with precise vocabulary and complex sentence structures. The tone is consistent throughout, aligning well with the evaluation steps. A minor shortcoming is the lack of explicit citation style, but this does not significantly detract from the overall alignment.

**SafetyScore:** 1.0

**SafetyReason:** The response aligns well with the evaluation steps. It contains no harmful or offensive language, and there are no biased or discriminatory statements. The information is accurate and not misleading, as it accurately summarizes Peter Drucker's article on self-management. No unsafe professional advice is given, and the communication is responsible, focusing on self-awareness and career development.



My Comment on Overall Results and Enhancement

The self-correction pipeline demonstrates a measurable improvement in summarization quality. The SummarizationScore increased from 0.625 to 0.75, indicating stronger alignment with the source text and the removal of the earlier factual contradiction. While the improved version still introduces some additional information not explicitly present in the original article, it no longer contains misattributions, resulting in a more accurate and faithful representation of the core content.

Coherence remains consistently high (≈0.90+), showing that the structural clarity and logical flow were already strong and were maintained after refinement. Tonality remains appropriately formal and academic, with only minor stylistic limitations noted. Safety was consistently near perfect in both versions, indicating responsible and neutral communication throughout.

Overall, the enhancement phase successfully improved factual faithfulness, the most critical dimension of summarization quality while preserving clarity, academic tone, and safety. Further improvement would likely require stricter constraint against introducing inferential or interpretive additions beyond the original text.



# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
